In [15]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
import pandas as pd 
pd.options.display.max_rows = 8  
import requests
import parsel
import warnings
import csv
import random
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib notebook
plt.rc('figure',figsize=(11,130))
plt.rcParams['font.sans-serif'] = ['SimHei']   # 解决图表中无法显示中文的问题
#忽略警告
warnings.filterwarnings("ignore")

In [16]:
file1 = "../实验四数据/国泰安数据/国泰安企业数据.xlsx"
df1 = pd.read_excel(file1,index_col=0)
df1

,证券代码,截止日期,专利产出,发明产出,实用新型产出,外观设计产出,专利授权(平均年限),发明授权(平均年限),实用新型授权(平均年限),外观设计授权(平均年限),...,外观设计终止(平均年限),ROA,ROE,资产负债率,营业总收入,无形资产净额,资产总计,政府资助,地区,行业名称
0,8,2016,20,0,20,0,1.529,3.346,0.439,0.000,...,0.000,0.064758,0.085953,0.246583,4.652829e+06,1.438561e+07,6.114151e+09,NaN,北京,NaN
1,9,2017,160,2,138,20,1.129,2.808,0.690,0.529,...,2.601,0.011924,0.032501,0.633121,2.891348e+06,5.966667e+04,1.236891e+10,646820.00,深圳,NaN
2,9,2007,10,9,1,0,0.971,0.000,0.971,0.000,...,3.721,0.056781,0.138924,0.435163,1.948492e+09,1.508513e+08,5.839300e+09,8725100.00,深圳,NaN
3,9,2010,34,15,10,9,1.584,2.768,0.686,0.711,...,2.110,0.003078,0.011107,0.722870,3.216777e+09,1.752473e+08,1.002465e+10,160000.00,深圳,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7112,603993,2016,28,3,25,0,0.525,0.000,0.525,0.000,...,0.000,0.011563,0.029683,0.514008,2.267260e+09,4.320426e+08,3.573702e+10,1171234.66,河南,有色金属矿采选业
7113,603996,2017,30,0,8,22,0.585,0.000,0.654,0.528,...,1.573,0.031513,0.097600,0.677122,6.201164e+09,4.547697e+07,4.394465e+09,9256223.64,浙江,信息技术业
7114,603997,2016,28,2,26,0,0.873,2.593,0.504,0.000,...,3.643,0.129746,0.162414,0.201140,1.380090e+09,4.480069e+07,1.840919e+09,4330000.00,浙江,交通运输设备制造业
7115,603998,2017,0,0,0,0,2.529,2.997,0.000,0.655,...,0.000,0.047069,0.062284,0.244282,5.248289e+08,8.074775e+07,1.274708e+09,400000.00,湖南,医药制造业


In [17]:
price_df = pd.read_excel("../实验四数据/国泰安数据/xlsx/市值.xlsx",index_col=0)
price_df
price_df = price_df.stack().reset_index()
price_df
price_df.info()
price_df.columns = ['证券代码','截止日期','市值']
price_df['证券代码'] = price_df['证券代码'].str[:-3].astype(int)
price_df

,2013,2011,2012,2014,2015,2016,2018,2010,2009,2017
证券代码,,,,,,,,,,
000001.SZ,1.004177e+11,7.987303e+10,8.207607e+10,1.809703e+11,1.715610e+11,1.562507e+11,1.610585e+11,5.502837e+10,7.567942e+10,2.283665e+11
000002.SZ,8.845020e+10,8.213422e+10,1.112750e+11,1.534214e+11,2.699909e+11,2.268546e+11,2.629526e+11,9.038063e+10,1.188582e+11,3.428761e+11
000004.SZ,9.783284e+08,6.793714e+08,6.869293e+08,1.307517e+09,3.854530e+09,3.770553e+09,1.346146e+09,1.011079e+09,8.397668e+08,1.879398e+09
000005.SZ,2.285834e+09,3.529328e+09,2.715571e+09,3.748768e+09,1.062771e+10,7.155709e+09,2.836879e+09,3.355604e+09,5.504288e+09,4.382343e+09
...,...,...,...,...,...,...,...,...,...,...
688777.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688788.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688981.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689009.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,证券代码,level_1,0
0,000001.SZ,2013,1.004177e+11
1,000001.SZ,2011,7.987303e+10
2,000001.SZ,2012,8.207607e+10
3,000001.SZ,2014,1.809703e+11
...,...,...,...
26048,603999.SH,2015,1.411200e+10
26049,603999.SH,2016,8.444160e+09
26050,603999.SH,2018,2.787840e+09
26051,603999.SH,2017,4.429440e+09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26052 entries, 0 to 26051
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   证券代码     26052 non-null  object 
 1   level_1  26052 non-null  object 
 2   0        26052 non-null  float64
dtypes: float64(1), object(2)
memory usage: 610.7+ KB


,证券代码,截止日期,市值
0,1,2013,1.004177e+11
1,1,2011,7.987303e+10
2,1,2012,8.207607e+10
3,1,2014,1.809703e+11
...,...,...,...
26048,603999,2015,1.411200e+10
26049,603999,2016,8.444160e+09
26050,603999,2018,2.787840e+09
26051,603999,2017,4.429440e+09


In [18]:
price_df = price_df.sort_values(by=['证券代码','截止日期'])
price_df.reset_index(inplace=True,drop=True)
price_df

,证券代码,截止日期,市值
0,1,2009,7.567942e+10
1,1,2010,5.502837e+10
2,1,2011,7.987303e+10
3,1,2012,8.207607e+10
...,...,...,...
26048,603999,2015,1.411200e+10
26049,603999,2016,8.444160e+09
26050,603999,2018,2.787840e+09
26051,603999,2017,4.429440e+09


In [19]:
df2 = pd.read_excel("../实验四数据/国泰安数据/xlsx/公司成立日期.xlsx")
df2

,证券代码,公司成立日期
0,100,2002-04-19
1,166,1996-09-16
2,333,2000-04-07
3,338,2002-12-23
...,...,...
2602,603996,2007-05-23
2603,603997,2003-07-11
2604,603998,2000-06-15
2605,603999,2009-12-28


In [20]:
df = pd.merge(price_df, df1, left_on=['证券代码','截止日期'], right_on=['证券代码','截止日期'])
df = pd.merge(df2, df,left_on='证券代码',right_on='证券代码')
df = pd.merge(df, df[df['截止日期']==2017]['证券代码'])
df

,证券代码,公司成立日期,截止日期,市值,专利产出,发明产出,实用新型产出,外观设计产出,专利授权(平均年限),发明授权(平均年限),...,外观设计终止(平均年限),ROA,ROE,资产负债率,营业总收入,无形资产净额,资产总计,政府资助,地区,行业名称


In [ ]:
# df['专利产出'] = df['专利授权(平均年限)']/df['资产总计']
# df['Age'] = (df['截止日期']-df['公司成立日期'].str[0:4].astype(int)).apply(np.log)
# df['公司规模'] = df['资产总计'].apply(np.log)
# df['Q'] = (df['市值']+df['负债合计']) / (df['资产总计'] - df['长期待摊费用'] - df['无形资产净额'] - df['递延所得税资产'])
# dummies = {'民营':0, '国企,民营':0, '民营,外资':0, '外资':0, '其他':0, np.nan:0, '国企':1}
# df['国有企业'] = df['股权性质'].map(dummies)
# df.head()

,证券代码,公司成立日期,截止日期,市值,专利产出,发明产出,实用新型产出,外观设计产出,专利授权(平均年限),发明授权(平均年限),...,ROE,资产负债率,营业总收入,无形资产净额,资产总计,政府资助,地区,行业名称,Age,公司规模


In [ ]:
df.columns